In [1]:
from pynq_dpu import DpuOverlay
from time import time
import numpy as np
from six.moves import urllib
import csv
import os
from itertools import product


model_num = 0
with open("model_result_GoogLeNet.csv","w") as csvfile: 
    writer = csv.writer(csvfile)
    writer.writerow(["index","input_horizontal","input_vertical","step","Execution time","Throughout"])

path = "./xmodel_GoogLeNet" #文件夹目录
files= os.listdir(path) #得到文件夹下的所有文件名称


input_shape_range =range(224,512,34)
            
for shape_x in input_shape_range:
    shape_y = shape_x
    file = "GoogLeNet_{input_x}_{input_y}.xmodel".format(input_x=shape_x,input_y=shape_y)
    if file in files: #遍历文件夹
        model_name = str(file)
        if not os.path.isdir(file): #判断是否是文件夹，不是文件夹才打开
            #Prepare the overlay
            overlay = DpuOverlay("dpu.bit")
            overlay.load_model("./xmodel_GoogLeNet/{}".format(file))

            opener = urllib.request.build_opener()
            opener.addheaders = [('User-agent', 'Mozilla/5.0')]
            urllib.request.install_opener(opener)

            #load test data

            test_data = np.random.randint(0,255,size=[50,shape_x,shape_y,1])
            test_label = np.random.randint(0,10,size=[50,])
            test_data = test_data.astype('float32')
            test_data /= 255.0
            print("Total number of test images: {}".format(test_data.shape[0]))
            print("  Dimension of each picture: {}x{}".format(test_data.shape[1],
                                                  test_data.shape[2]))

            #use VART 
            dpu = overlay.runner

            inputTensors = dpu.get_input_tensors()
            outputTensors = dpu.get_output_tensors()

            shapeIn = tuple(inputTensors[0].dims)
            shapeOut = tuple(outputTensors[0].dims)
            outputSize = int(outputTensors[0].get_data_size() / shapeIn[0])

            softmax = np.empty(outputSize)

            output_data = [np.empty(shapeOut, dtype=np.float32, order="C")]
            input_data = [np.empty(shapeIn, dtype=np.float32, order="C")]
            image = input_data[0]

            def calculate_softmax(data):
                result = np.exp(data)
                return result


            #Run DPU to make predictions

            total = test_data.shape[0]
            predictions = np.empty_like(test_label)
            print("Classifying {} digit pictures ...".format(total))

            start = time()
            for i in range(total):
                image[0,...] = test_data[i]
                job_id = dpu.execute_async(input_data, output_data)
                dpu.wait(job_id)
                temp = [j.reshape(1, outputSize) for j in output_data]
                softmax = calculate_softmax(temp[0][0])
                predictions[i] = softmax.argmax()

            stop = time()
            correct = np.sum(predictions==test_label)
            execution_time = (stop-start)*20
            print("Overall accuracy: {}".format(correct/total))
            print("  Execution time/per picture: {:.4f}ms".format(execution_time))
            print("      Throughput: {:.4f}FPS".format(total/execution_time))

#             lo_Conv2D_kernel = model_name.find("Conv2D_kernel")
#             lo_Conv2D_size = model_name.find("Conv2D_size")
#             lo_MaxPooling_size = model_name.find("MaxPooling_size")
#             lo_Dense_size = model_name.find("Dense_size")
#             lo_end = model_name.find(".xmodel")
            with open("model_result_GoogLeNet.csv","a+") as csvfile: 
                writer = csv.writer(csvfile)
                writer.writerows([[model_num,shape_x,shape_y,step,execution_time,total/execution_time],])
                model_num += 1


            #clean up 
            del overlay
            del dpu

In [2]:
from pynq_dpu import DpuOverlay
from time import time
import numpy as np
from six.moves import urllib
import csv
import os
from itertools import product


model_num = 0
with open("model_result_GoogLeNet.csv","w") as csvfile: 
    writer = csv.writer(csvfile)
    writer.writerow(["index","input_horizontal","input_vertical","step","Execution time","Throughout"])

path = "./xmodel_GoogLeNet" #文件夹目录
files= os.listdir(path) #得到文件夹下的所有文件名称


input_shape_range =range(32,512,2)
step_range =range(1,11,1)            
for shape_x,step in product(input_shape_range,step_range):
    shape_y = shape_x
    file = "GoogLeNet_{input_x}_{input_y}_{step}.xmodel".format(input_x=shape_x,input_y=shape_y,step=step)
    if file in files: #遍历文件夹
        model_name = str(file)
        if not os.path.isdir(file): #判断是否是文件夹，不是文件夹才打开
            #Prepare the overlay
            overlay = DpuOverlay("dpu.bit")
            overlay.load_model("./xmodel_GoogLeNet/{}".format(file))

            opener = urllib.request.build_opener()
            opener.addheaders = [('User-agent', 'Mozilla/5.0')]
            urllib.request.install_opener(opener)

            #load test data

            test_data = np.random.randint(0,255,size=[25,shape_x,shape_y,1])
            test_label = np.random.randint(0,10,size=[25,])
            test_data = test_data.astype('float32')
            test_data /= 255.0
            print("Total number of test images: {}".format(test_data.shape[0]))
            print("  Dimension of each picture: {}x{}".format(test_data.shape[1],
                                                              test_data.shape[2]))

            #use VART 
            dpu = overlay.runner

            inputTensors = dpu.get_input_tensors()
            outputTensors = dpu.get_output_tensors()

            shapeIn = tuple(inputTensors[0].dims)
            shapeOut = tuple(outputTensors[0].dims)
            outputSize = int(outputTensors[0].get_data_size() / shapeIn[0])

            softmax = np.empty(outputSize)

            output_data = [np.empty(shapeOut, dtype=np.float32, order="C")]
            input_data = [np.empty(shapeIn, dtype=np.float32, order="C")]
            image = input_data[0]

            def calculate_softmax(data):
                result = np.exp(data)
                return result


            #Run DPU to make predictions

            total = test_data.shape[0]
            predictions = np.empty_like(test_label)
            print("Classifying {} digit pictures ...".format(total))

            start = time()
            for i in range(total):
                image[0,...] = test_data[i]
                job_id = dpu.execute_async(input_data, output_data)
                dpu.wait(job_id)
                temp = [j.reshape(1, outputSize) for j in output_data]
                softmax = calculate_softmax(temp[0][0])
                predictions[i] = softmax.argmax()

            stop = time()
            correct = np.sum(predictions==test_label)
            execution_time = (stop-start)*40
            print("Overall accuracy: {}".format(correct/total))
            print("  Execution time/per picture: {:.4f}ms".format(execution_time))
            print("      Throughput: {:.4f}FPS".format(total/execution_time))

#             lo_Conv2D_kernel = model_name.find("Conv2D_kernel")
#             lo_Conv2D_size = model_name.find("Conv2D_size")
#             lo_MaxPooling_size = model_name.find("MaxPooling_size")
#             lo_Dense_size = model_name.find("Dense_size")
#             lo_end = model_name.find(".xmodel")
            with open("model_result_GoogLeNet.csv","a+") as csvfile: 
                writer = csv.writer(csvfile)
                writer.writerows([[model_num,shape_x,shape_y,step,execution_time,total/execution_time],])
                model_num += 1


            #clean up 
            del overlay
            del dpu

IndexError: list index out of range

In [1]:
            del overlay
            del dpu

NameError: name 'overlay' is not defined